# Publish an entire folder as an OER "book"

In [ ]:
!rm -rf bookfiles

In [ ]:
!mkdir -p bookfiles

In [2]:
from pathlib import Path
from jupyterinstruct import InstructorNotebook as inb
from jupyterinstruct.nbfilename import nbfilename
from jupyterinstruct.nbvalidate import validate
from jupyterinstruct import webtools 
from nbconvert.preprocessors import ExecutePreprocessor

<IPython.core.display.Javascript object>

In [3]:
import thisbook

tags = thisbook.tags()

In [4]:
# jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace $nbfile
# jupyter nbconvert --execute --no-prompt --allow-errors --to html $nbfile
# #	jupyter nbconvert $nbfile --to html
# cp $nbfile $webdir 
# newfile=`basename $nbfile .ipynb`
# cp ${newfile}.html $webdir 

In [5]:
def makechapter(filename,tags):
    print(filename)
    nb = inb.InstructorNB(filename=filename)

    nb.removebefore('ENDHEADER')
    nb.removeafter('STARTFOOTER')
    nb.headerfooter('Book-Header.ipynb', 'Book-Footer.ipynb')
    nb.removecells('###MSU###')
    studentfile = nb.makestudent(tags,studentfolder="./bookfiles/")
    nb.writenotebook(studentfile)
    validate(studentfile)
   
    #done = webtools.publish2folder(studentfile, website_folder="./bookfiles/")
    #display(done)
    return studentfile

In [6]:
makechapter(Path('../0-Introduction_INSTRUCTOR.ipynb'),tags);

../0-Introduction_INSTRUCTOR.ipynb
Myfilename ../0-Introduction_INSTRUCTOR.ipynb


0-Introduction.ipynb


Validating Notebook ./bookfiles/0-Introduction.ipynb
   ERROR: Image LINK not found - http://creativecommons.org/licenses/by-nc/4.0/
   ERROR: Image LINK not found - http://creativecommons.org/licenses/by-nc/4.0/


In [ ]:
#Make a temporary folder and convert all Instructor
#notebooks to student notebooks for testing

directory = Path('..')

allfiles = sorted(directory.glob('*.ipynb'))
allfiles.sort
#Make student versions
for filename in allfiles:
    nbfile = nbfilename(filename)
    if not 'Quiz' in str(nbfile): # Skip quiz prep
        if nbfile.isInstructor: # Only process INSTRUCTOR notebooks
            if nbfile.prefix.isdigit(): # Only process numrical notebooks
                filename = makechapter(filename, tags)

In [ ]:
fid = open('./bookfiles/_toc.yml', 'w')
directory = Path('./bookfiles/')

allfiles = sorted(directory.glob('*.ipynb'))

for ipynb in allfiles:
    if not '0000' in str(ipynb):
        print(ipynb.name)
        fid.write(f'- file: {ipynb.name}\n')
    
for ipynb in allfiles:
    if '0000' in str(ipynb):
        print(ipynb.name)
        fid.write(f'- file: {ipynb.name}\n')
    
fid.close()

In [7]:
!cp Cover_horizontal.png ./bookfiles/
!cp Cover.jpg ./bookfiles/
!cp _config.yml ./bookfiles/

In [8]:
!jupyter-book build ./bookfiles/

Running Jupyter-Book v0.10.2
Source Folder: /Users/colbrydi/REPOS/MTH314_Textbook/MakeTextbook/bookfiles
Config Path: /Users/colbrydi/REPOS/MTH314_Textbook/MakeTextbook/bookfiles/_config.yml
Output Path: /Users/colbrydi/REPOS/MTH314_Textbook/MakeTextbook/bookfiles/_build/html
Running Sphinx v3.5.4
loading pickled environment... done
myst v0.13.7: MdParserConfig(renderer='sphinx', commonmark_only=False, dmath_allow_labels=True, dmath_allow_space=True, dmath_allow_digits=True, update_mathjax=True, enable_extensions=['colon_fence', 'dollarmath', 'linkify', 'substitution'], disable_syntax=[], url_schemes=['mailto', 'http', 'https'], heading_anchors=None, html_meta=[], footnote_transition=True, substitutions=[], sub_delimiters=['{', '}'])
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 1 source files that are out of date
updating environment: 0 added, 1 changed, 0 removed
Did not execute 0-Introduction. Set jupyter_execute_notebooks to `force` to exec

In [9]:
#!open ./bookfiles/_build/html/index.html

In [10]:
!cp -rf ./bookfiles/_build/html/* ./docs/

In [11]:
!open ./docs/index.html